In [13]:
!pip install bark ffprobe ffmpeg pydub

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for ffprobe: filename=ffprobe-0.5-py3-none-any.whl size=3405 sha256=66e281df04e7efce11de3322198c5fd23968fb8fffddfb039e3ecf459ca3cd5b
  Stored in directory: /root/.cache/pip/wheels/a2/66/e3/5da9a7e12ee519eed653b188eb8dd7ca780f5c882922beb15d
  Created wheel for ffmpeg: filename=ffmpeg-1.4-py3-none-any.whl size=6083 sha256=751d3a89aa57c3698a7b7896d11861a702d52df49365b2bb4d790b7d9b660c5e
  Stored in directory: /root/.cache/pip/wheels/8e/7a/69/cd6aeb83b126a7f04cbe7c9d929028dc52a6e7d525ff56003a
Successfully built ffprobe ffmpeg


In [15]:
from pydub import AudioSegment

m4a_file = '/kaggle/input/prompt2/Prompt1.m4a' # I have downloaded sample audio from this link https://getsamplefiles.com/sample-audio-files/m4a
wav_filename = '/kaggle/working/output.wav'

sound = AudioSegment.from_file(m4a_file, format='m4a')
file_handle = sound.export(wav_filename, format='wav')

In [23]:
from bark.generation import load_codec_model, generate_text_semantic
from encodec.utils import convert_audio

import torchaudio
import torch
import numpy as np

device = 'cuda' # or 'cpu'
model = load_codec_model(use_gpu=True if device == 'cuda' else False)

In [3]:
device

'cuda'

In [4]:
!git clone https://github.com/gitmylo/bark-voice-cloning-HuBERT-quantizer/
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu117

Cloning into 'bark-voice-cloning-HuBERT-quantizer'...
remote: Enumerating objects: 1882, done.
remote: Counting objects: 100% (247/247), done.
remote: Compressing objects: 100% (123/123), done.
remote: Total 1882 (delta 141), reused 207 (delta 119), pack-reused 1635
Receiving objects: 100% (1882/1882), 319.75 MiB | 28.90 MiB/s, done.
Resolving deltas: 100% (142/142), done.
Looking in indexes: https://download.pytorch.org/whl/cu117


In [5]:
!pip install -r /kaggle/working/bark-voice-cloning-HuBERT-quantizer/requirements.txt

Ignoring soundfile: markers 'platform_system == "Windows"' don't match your environment
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 81.2 MB/s eta 0:00:00:00:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 819.1/819.1 kB 49.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.8/123.8 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.5/286.5 kB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.4/112.4 kB 11.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fairseq: filename=fairseq-0.12.2-cp310-cp310-linux_x86_64.whl size=10415054 sha256=f43093aa3d95644a49f05e0698f9b5814e9fc23f633b180d9d1a22da43c1935a
  Stored in directory: /root/.

In [6]:
import os
print(os.getcwd())
%cd '/kaggle/working/bark-voice-cloning-HuBERT-quantizer'
print(os.getcwd())

/kaggle/working
/kaggle/working/bark-voice-cloning-HuBERT-quantizer
/kaggle/working/bark-voice-cloning-HuBERT-quantizer


In [7]:
from bark_hubert_quantizer.hubert_manager import HuBERTManager
hubert_manager = HuBERTManager()
hubert_manager.make_sure_hubert_installed()
hubert_manager.make_sure_tokenizer_installed()

Downloaded HuBERT


Downloaded tokenizer


'data/models/hubert/tokenizer.pth'

In [8]:
from bark_hubert_quantizer.pre_kmeans_hubert import CustomHubert
from bark_hubert_quantizer.customtokenizer import CustomTokenizer

# Load the HuBERT model
hubert_model = CustomHubert(checkpoint_path='data/models/hubert/hubert.pt').to(device)

# Load the CustomTokenizer model
tokenizer = CustomTokenizer.load_from_checkpoint('data/models/hubert/tokenizer.pth').to(device)

In [16]:
audio_filepath = '/kaggle/working/output.wav' # the audio you want to clone (under 13 seconds)
wav, sr = torchaudio.load(audio_filepath)
wav = convert_audio(wav, sr, model.sample_rate, model.channels)
wav = wav.to(device)

In [17]:
semantic_vectors = hubert_model.forward(wav, input_sample_hz=model.sample_rate)
semantic_tokens = tokenizer.get_token(semantic_vectors)

In [18]:
with torch.no_grad():
    encoded_frames = model.encode(wav.unsqueeze(0))
codes = torch.cat([encoded[0] for encoded in encoded_frames], dim=-1).squeeze()  # [n_q, T]

In [19]:
codes = codes.cpu().numpy()
# move semantic tokens to cpu
semantic_tokens = semantic_tokens.cpu().numpy()

In [24]:
voice_name = 'voice_output' # whatever you want the name of the voice to be
output_path = '/kaggle/working/' + voice_name + '.npz'
np.savez(output_path, fine_prompt=codes, coarse_prompt=codes[:2, :], semantic_prompt=semantic_tokens)

In [25]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0"


from IPython.display import Audio
import nltk  # we'll use this to split into sentences
import numpy as np

from bark.generation import (
    generate_text_semantic,
    preload_models,
)
from bark.api import semantic_to_waveform
from bark import generate_audio, SAMPLE_RATE

In [ ]:
preload_models()

In [ ]:
script = """
Hey this is nandan trying to clone the voice in prompt.
This is to improve counsello project and to reduce the 
human effort in creation of dataset. We need the dataset
of voices and thus we are using text to speech, this dataset
will be used to finetune whisper model which is voice to 
text model.
""".replace("\n", " ").strip()

In [29]:
sentences = nltk.sent_tokenize(script)

In [31]:
GEN_TEMP = 0.6
SPEAKER = "/kaggle/working/voice_output.npz"
silence = np.zeros(int(0.25 * SAMPLE_RATE))  # quarter second of silence

pieces = []
for sentence in sentences:
    semantic_tokens = generate_text_semantic(
        sentence,
        history_prompt=SPEAKER,
        temp=GEN_TEMP,
        min_eos_p=0.05,  # this controls how likely the generation is to end
    )

    audio_array = semantic_to_waveform(semantic_tokens, history_prompt=SPEAKER,)
    pieces += [audio_array, silence.copy()]

100%|██████████| 36/36 [00:28<00:00,  1.28it/s]


In [ ]:
Audio(np.concatenate(pieces), rate=SAMPLE_RATE)